In [1]:
import pandas as pd
import numpy as np
import pathlib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from model.threshold_guess import compute_thresholds, cut
from model.gosdt import GOSDT

# Example 1
GOSDT runs on binary datasets. In this example, we run GOSDT on a binarized Monk 1 dataset. We download Monk 1 from the UCI data repository and use one-hot encoding from the Sklearn package to transform categorical features to binary features. 

In [2]:
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/monks-problems/monks-1.train", sep=" ", header=None)
df = df.iloc[:,1:-1]

# assign column names 
df.columns = ["label", "head_shape", "body_shape", "is_smiling", "holding", "jacket_color", "has_tie"]
X = df.iloc[:,1:]
y = df.iloc[:,0]

# Encode the categorical features to binary features
enc = OneHotEncoder()
enc.fit(X)
X_new = pd.DataFrame(enc.transform(X).toarray(), dtype=int)
X_new.columns = ["head_shape_round", "head_shape_square", "head_shape_octagon", 
                 "body_shape_round", "body_shape_square", "body_shape_octagon",
                 "is_smiling_yes", "is_smiling_no", 
                 "holding_sword", "holding_balloon", "holding_flag",
                 "jacket_color_red", "jacket_color_yellow", "jacket_color_green", "jacket_color_blue",
                 "has_tie_yes", "has_tie_no"
                ]
X_new

,head_shape_round,head_shape_square,head_shape_octagon,body_shape_round,body_shape_square,body_shape_octagon,is_smiling_yes,is_smiling_no,holding_sword,holding_balloon,holding_flag,jacket_color_red,jacket_color_yellow,jacket_color_green,jacket_color_blue,has_tie_yes,has_tie_no
0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0
1,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1
2,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0
3,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1
4,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,1
120,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1
121,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1
122,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1


Set GOSDT configuration and fit model. GOSDT finds a 7-leaf tree with 100% accuracy within seconds. 

In [3]:
# set GOSDT configuration
config = {
            "regularization": 0.02, # regularization penalizes the tree with more leaves. We recommend to set it to relative high value to find a sparse tree.  
            "depth_budget": 6,
            "time_limit": 60, # training time limit in seconds
            "similar_support": False
        }

model = GOSDT(config)

# train GOSDT 
model.fit(X_new, y)
print("evaluate the model, extracting tree and scores", flush=True)

acc = model.score(X_new, y)
n_leaves = model.leaves()
n_nodes = model.nodes()
time = model.utime

print("Model training time: {}".format(time))
print("Training accuracy: {}".format(acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

gosdt reported successful execution
training completed. 0.000/0.000/0.571 (user, system, wall), mem=0 MB
bounds: [0.140000..0.140000] (0.000000) loss=0.000000, iterations=22433
evaluate the model, extracting tree and scores
Model training time: 0.0
Training accuracy: 1.0
# of leaves: 7
if jacket_color_red = 1 then:
    predicted class: 1
    misclassification penalty: 0.0
    complexity penalty: 0.02

else if body_shape_round = 1 and head_shape_round = 1 and jacket_color_red != 1 then:
    predicted class: 1
    misclassification penalty: 0.0
    complexity penalty: 0.02

else if body_shape_round != 1 and head_shape_round = 1 and jacket_color_red != 1 then:
    predicted class: 0
    misclassification penalty: 0.0
    complexity penalty: 0.02

else if body_shape_square = 1 and head_shape_round != 1 and head_shape_square = 1 and jacket_color_red != 1 then:
    predicted class: 1
    misclassification penalty: 0.0
    complexity penalty: 0.02

else if body_shape_square != 1 and head_shap

# Example 2
In our first example, we showed that GOSDT can solve a small dataset (Monk 1) in seconds. In this example, we run GOSDT on COMPAS, a recidivism dataset. The COMPAS dataset is much larger, containing 6907 samples and 7 continuous features.  

In [4]:
# read the dataset
df = pd.read_csv("../experiments/datasets/compas.csv")
X, y = df.iloc[:,:-1], df.iloc[:,-1]
h = df.columns[:-1]
df

,sex=female,age,juv_fel_count,juv_misd_count,juvenile_crimes,priors_count,current_charge_degree=felony,two_year_recid
0,0,69,0,0,0,0,1,0
1,0,34,0,0,0,0,1,1
2,0,24,0,0,1,4,1,1
3,0,44,0,0,0,0,0,0
4,0,41,0,0,0,14,1,1
...,...,...,...,...,...,...,...,...
6902,0,23,0,0,0,0,1,0
6903,0,23,0,0,0,0,1,0
6904,0,57,0,0,0,0,1,0
6905,1,33,0,0,0,3,0,0


To start, we fit a simple model with a small depth constraint and large regularization. Even for a complicated dataset, GOSDT can efficiently find an optimal tree. 

In [5]:
# train GOSDT model
config = {
            "regularization": 0.01, # regularization penalizes the tree with more leaves. We recommend to set it to relative high value to find a sparse tree.  
            "depth_budget": 4,
            "time_limit": 60, # training time limit in seconds
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X, y)
print("evaluate the model, extracting tree and scores", flush=True)

acc = model.score(X, y)
n_leaves = model.leaves()

print("Training accuracy: {}".format(acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

gosdt reported successful execution
training completed. 0.000/0.000/15.097 (user, system, wall), mem=0 MB
bounds: [0.366760..0.366760] (0.000000) loss=0.336760, iterations=37181
evaluate the model, extracting tree and scores
Training accuracy: 0.6632401911104676
# of leaves: 3
if 23 <= age and 3 <= priors_count then:
    predicted class: 1
    misclassification penalty: 0.14
    complexity penalty: 0.01

else if 23 <= age and priors_count < 3 then:
    predicted class: 0
    misclassification penalty: 0.15
    complexity penalty: 0.01

else if age < 23 then:
    predicted class: 1
    misclassification penalty: 0.046
    complexity penalty: 0.01


In the above, we had to constrain our search to very simple trees. We're likely to get a more useful and accurate model when we allow our model to find more complicated trees (we do this with less constraint on depth and with smaller regularization). However, for real-valued datasets like COMPAS, GOSDT might not finish running within a couple minutes. An example is shown below where the depth budget is set to 6 and the regularization is set to 0.001 for the COMPAS dataset. After 60 seconds, GOSDT hits the time limit and returns the current best model. 

In [6]:
# set GOSDT configuration
# set a smaller regularization and a higher depth budget. All other hyparameters are same as before. 
config = {
            "regularization": 0.001, 
            "depth_budget": 6,
            "time_limit": 60, 
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X, y)
print("evaluate the model, extracting tree and scores", flush=True)

acc = model.score(X, y)
n_leaves = model.leaves()

print("Training accuracy: {}".format(acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

gosdt reported possible timeout.
training completed. -1.000/-1.000/-1.000 (user, system, wall), mem=0 MB
bounds: [0.225962..0.356857] (0.130895) loss=0.333285, iterations=29999
evaluate the model, extracting tree and scores
Training accuracy: 0.6667149268857681
# of leaves: 4
if 56 <= age and 3 <= priors_count then:
    predicted class: 0
    misclassification penalty: 0.012
    complexity penalty: 0.001

else if age < 56 and 3 <= priors_count then:
    predicted class: 1
    misclassification penalty: 0.127
    complexity penalty: 0.001

else if 23 <= age and priors_count < 3 then:
    predicted class: 0
    misclassification penalty: 0.15
    complexity penalty: 0.001

else if age < 23 and priors_count < 3 then:
    predicted class: 1
    misclassification penalty: 0.044
    complexity penalty: 0.001


With the time-out, GOSDT didn't find a very accurate tree because it couldn't finish optimizing. If it did finish optimizing, we might need to wait for a long time or use a large amount of memory. An alternative, presented in [our AAAI '22 paper](https://arxiv.org/abs/2112.00798), is to use threshold and lowerbound guesses to help GOSDT find a near optimal tree for COMPAS within 1 minute. The guessing process works as follows: 

#### 1. Threshold guess: we first use a black box model, gradient boosted trees, to guess useful thresholds. 

One of the biggest slowdowns for decision trees is deciding how to binarize the continuous features. We had a separate column in our dataset for age <=20.5, age <=21.5, age <= 22.5, ... and so on. Using threshold guessing lets us prune the thresholds we use, by looking at which thresholds are used by gradient boosted trees. 

In [7]:
# GBDT parameters for threshold and lower bound guesses
n_est = 40
max_depth = 1

In [8]:
# guess thresholds
X_train = pd.DataFrame(X, columns=h)
X_train_guessed, thresholds, header, threshold_guess_time = compute_thresholds(X_train.copy(), y, n_est, max_depth)
X_train_guessed

,age<=20.5,age<=22.5,age<=27.5,age<=29.5,age<=33.5,age<=34.5,age<=36.5,juvenile_crimes<=0.5,priors_count<=0.5,priors_count<=1.5,priors_count<=2.5,priors_count<=3.5,priors_count<=5.5,priors_count<=7.5,priors_count<=8.5,age<=23.5
0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0
1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0
2,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,0
3,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6902,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6903,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6904,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0
6905,0,0,0,0,1,1,1,1,0,0,0,1,1,1,1,0


#### 2. Lower bound guess: we use the gradient boosted tree to get labels for the training dataset. 

We determine what labels the gradient boosted tree model would predict for each of the training points. The idea is that many good decision trees are likely to make similar predictions to the gradient boosted tree on the training data.

In [9]:
# guess lower bound
import time
start_time = time.perf_counter()
clf = GradientBoostingClassifier(n_estimators=n_est, max_depth=max_depth, random_state=42)
clf.fit(X_train_guessed, y.values.flatten())
warm_labels = clf.predict(X_train_guessed)

elapsed_time = time.perf_counter() - start_time

lb_time = elapsed_time

In [10]:
# save the labels as a tmp file and return the path to it.
labelsdir = pathlib.Path('/tmp/warm_lb_labels')
labelsdir.mkdir(exist_ok=True, parents=True)

labelpath = labelsdir / 'warm_label.tmp'
labelpath = str(labelpath)
pd.DataFrame(warm_labels).to_csv(labelpath, header="class_labels",index=None) # TODO: verify this formats correctly for gosdt (shouldn't require headers)


#### 3. train GOSDT model

We can use the information from our guesses to speed up training GOSDT, while retaining some nice properties on the tree it returns (see our AAAI '22 paper for more information). 

In [11]:
# train GOSDT model
config = {
            "regularization": 0.001, 
            "depth_budget": 6,
            "time_limit": 60, 
            "warm_LB": True,
            "path_to_labels": labelpath,
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X_train_guessed, pd.DataFrame(y))

print("evaluate the model, extracting tree and scores", flush=True)

gosdt reported successful execution
training completed. 0.000/0.000/0.883 (user, system, wall), mem=0 MB
bounds: [0.322898..0.322898] (0.000000) loss=0.314898, iterations=11685
evaluate the model, extracting tree and scores


#### 4. Get results

In [12]:
# get the results
acc = model.score(X_train_guessed, y)
n_leaves = model.leaves()

print("Training accuracy: {}".format(acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

Training accuracy: 0.6851020703633994
# of leaves: 8
if age<=20.5 = 1 and age<=27.5 = 1 and juvenile_crimes<=0.5 = 1 then:
    predicted class: 1
    misclassification penalty: 0.005
    complexity penalty: 0.001

else if age<=20.5 != 1 and age<=27.5 = 1 and juvenile_crimes<=0.5 = 1 and priors_count<=1.5 = 1 then:
    predicted class: 0
    misclassification penalty: 0.06
    complexity penalty: 0.001

else if age<=20.5 != 1 and age<=27.5 = 1 and juvenile_crimes<=0.5 = 1 and priors_count<=1.5 != 1 then:
    predicted class: 1
    misclassification penalty: 0.03
    complexity penalty: 0.001

else if age<=27.5 = 1 and juvenile_crimes<=0.5 != 1 then:
    predicted class: 1
    misclassification penalty: 0.023
    complexity penalty: 0.001

else if age<=27.5 != 1 and age<=36.5 = 1 and priors_count<=2.5 = 1 then:
    predicted class: 0
    misclassification penalty: 0.046
    complexity penalty: 0.001

else if age<=27.5 != 1 and age<=36.5 = 1 and priors_count<=2.5 != 1 then:
    predicted 

After using the threshold and lower bound guesses, GOSDT can find a near optimal tree within seconds. This tree is much more accurate than the tree we found earlier, when using much more restrictive parameters. 

# Example 3
In this example, we show a comprehensive modeling and evaluation process using GOSDT with guesses on the COMPAS dataset. This example includes how to split data into training and test set and how to get training and test set after threshold guessing. 

For this example, we start by using guessing immediately, which is what we recommend you do if you have an extraordinarily large number of features. We _could_ have followed the same process as in example 2 (first run without guessing, then see what happens and use guessing if we time out). However, there are so many features here that we will definitely need guessing to find accurate trees in reasonable time and memory. If we tried running without guessing, we would be likely to kill the kernel because of how much memory we would be using!

In [13]:
# read the dataset
df = pd.read_csv("../experiments/datasets/compas.csv")
X, y = df.iloc[:,:-1], df.iloc[:,-1]
h = df.columns[:-1]
df

,sex=female,age,juv_fel_count,juv_misd_count,juvenile_crimes,priors_count,current_charge_degree=felony,two_year_recid
0,0,69,0,0,0,0,1,0
1,0,34,0,0,0,0,1,1
2,0,24,0,0,1,4,1,1
3,0,44,0,0,0,0,0,0
4,0,41,0,0,0,14,1,1
...,...,...,...,...,...,...,...,...
6902,0,23,0,0,0,0,1,0
6903,0,23,0,0,0,0,1,0
6904,0,57,0,0,0,0,1,0
6905,1,33,0,0,0,3,0,0


In [14]:
# If train-test split is desired
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)
print(X_train.shape)
print(X_test.shape)

(5525, 7)
(1382, 7)


#### 1. Threshold guess: we first use the gradient boosted tree to guess useful thresholds. 

In [15]:
# GBDT parameters for threshold and lower bound guesses
n_est = 40
max_depth = 1

In [16]:
# guess thresholds
X_train = pd.DataFrame(X_train, columns=h)
X_test = pd.DataFrame(X_test, columns=h)
X_train_guessed, thresholds, header, threshold_guess_time = compute_thresholds(X_train.copy(), y_train, n_est, max_depth)
X_test_guessed = cut(X_test.copy(), thresholds)
X_test_guessed = X_test_guessed[header]
print(X_train_guessed.shape)
print(X_test_guessed.shape)
print("train set column names == test set column names: {}".format(list(X_train_guessed.columns)==list(X_test_guessed.columns)))

(5525, 17)
(1382, 17)
train set column names == test set column names: True


#### 2. Lower bound guess: we use the gradient boosted tree to get labels for the training dataset. 

In [17]:
# guess lower bound
import time
start_time = time.perf_counter()
clf = GradientBoostingClassifier(n_estimators=n_est, max_depth=max_depth, random_state=42)
clf.fit(X_train_guessed, y_train.values.flatten())
warm_labels = clf.predict(X_train_guessed)

elapsed_time = time.perf_counter() - start_time

lb_time = elapsed_time

In [18]:
# save the labels as a tmp file and return the path to it.
labelsdir = pathlib.Path('/tmp/warm_lb_labels')
labelsdir.mkdir(exist_ok=True, parents=True)

labelpath = labelsdir / 'warm_label.tmp'
labelpath = str(labelpath)
pd.DataFrame(warm_labels).to_csv(labelpath, header="class_labels",index=None) # TODO: verify this formats correctly for gosdt (shouldn't require headers)


#### 3. train GOSDT model

In [19]:
# train GOSDT model
config = {
            "regularization": 0.001,
            "depth_budget": 5,
            "time_limit": 60,
            "warm_LB": True,
            "path_to_labels": labelpath,
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X_train_guessed, pd.DataFrame(y_train))

print("evaluate the model, extracting tree and scores", flush=True)

gosdt reported successful execution
training completed. 0.000/0.000/0.242 (user, system, wall), mem=0 MB
bounds: [0.325914..0.325914] (0.000000) loss=0.318914, iterations=4226
evaluate the model, extracting tree and scores


#### 4. Get results

In [20]:
# get the results
train_acc = model.score(X_train_guessed, y_train)
test_acc = model.score(X_test_guessed, y_test)
n_leaves = model.leaves()
n_nodes = model.nodes()
time = model.utime

print("Model training time: {}".format(time))
print("Training accuracy: {}".format(train_acc))
print("Test accuracy: {}".format(test_acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

Model training time: 0.0
Training accuracy: 0.6810859728506787
Test accuracy: 0.6903039073806078
# of leaves: 7
if age<=21.5 = 1 then:
    predicted class: 1
    misclassification penalty: 0.026
    complexity penalty: 0.001

else if age<=21.5 != 1 and age<=27.5 = 1 and priors_count<=1.5 = 1 then:
    predicted class: 0
    misclassification penalty: 0.053
    complexity penalty: 0.001

else if age<=21.5 != 1 and age<=27.5 = 1 and priors_count<=1.5 != 1 then:
    predicted class: 1
    misclassification penalty: 0.04
    complexity penalty: 0.001

else if age<=21.5 != 1 and age<=27.5 != 1 and age<=38.5 = 1 and priors_count<=3.5 = 1 then:
    predicted class: 0
    misclassification penalty: 0.068
    complexity penalty: 0.001

else if age<=21.5 != 1 and age<=27.5 != 1 and age<=38.5 = 1 and priors_count<=3.5 != 1 then:
    predicted class: 1
    misclassification penalty: 0.045
    complexity penalty: 0.001

else if age<=21.5 != 1 and age<=27.5 != 1 and age<=38.5 != 1 and priors_count<=

As you can see, we find a reasonably accurate model quite quickly on datasets!

Thank you for reading our tutorial. Please do try out our methods with different parameters and datasets. Happy tree training!